In [1]:
import sys; sys.path.append('../src')

from input_output.parser import Parser
from input_output.writer import Writer
from preprocessing.tokenizer import tokenize
from models.rules import classify_token, classify_tokens
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 1000)

In [3]:
bank_type = 'NER'
bank_name = 'DrugBank'
#dir_type = f'Test-{bank_type}'
dir_type = f'Train'
bank_dir = f'../resources/{dir_type}/{bank_name}/'


In [4]:
df = Parser(bank_dir).call()

df['tokens'] = df['sentence'].apply(tokenize)

get_drugs = lambda tokens: [t for t in  [classify_token(t) for t in tokens] if t is not None]
df['drugs'] = df['tokens'].apply(get_drugs)

In [5]:
df.head()

,id,sentence,parsed_drugs,parsed_pairs,tokens,drugs
0,DDI-DrugBank.d436.s0,"No drug, nutritional supplement, food or herb interactions have yet been reported.",[],[],"[{'text': 'No', 'char_offset': [0, 1]}, {'text': 'drug', 'char_offset': [3, 6]}, {'text': ',', 'char_offset': [7, 7]}, {'text': 'nutritional', 'char_offset': [9, 19]}, {'text': 'supplement', 'char_offset': [21, 30]}, {'text': ',', 'char_offset': [31, 31]}, {'text': 'food', 'char_offset': [33, 36]}, {'text': 'or', 'char_offset': [38, 39]}, {'text': 'herb', 'char_offset': [41, 44]}, {'text': 'interactions', 'char_offset': [46, 57]}, {'text': 'have', 'char_offset': [59, 62]}, {'text': 'yet', 'char_offset': [64, 66]}, {'text': 'been', 'char_offset': [68, 71]}, {'text': 'reported', 'char_offset': [73, 80]}, {'text': '.', 'char_offset': [81, 81]}]",[<DrugEntity None 46-57 interactions group>]
1,DDI-DrugBank.d519.s0,No formal drug/drug interaction studies with Plenaxis were performed.,[<DrugEntity DDI-DrugBank.d519.s0.e0 45-52 Plenaxis brand>],[],"[{'text': 'No', 'char_offset': [0, 1]}, {'text': 'formal', 'char_offset': [3, 8]}, {'text': 'drug/drug', 'char_offset': [10, 18]}, {'text': 'interaction', 'char_offset': [20, 30]}, {'text': 'studies', 'char_offset': [32, 38]}, {'text': 'with', 'char_offset': [40, 43]}, {'text': 'Plenaxis', 'char_offset': [45, 52]}, {'text': 'were', 'char_offset': [54, 57]}, {'text': 'performed', 'char_offset': [59, 67]}, {'text': '.', 'char_offset': [68, 68]}]",[]
2,DDI-DrugBank.d519.s1,Cytochrome P-450 is not known to be involved in the metabolism of Plenaxis.,[<DrugEntity DDI-DrugBank.d519.s1.e0 66-73 Plenaxis brand>],[],"[{'text': 'Cytochrome', 'char_offset': [0, 9]}, {'text': 'P-450', 'char_offset': [11, 15]}, {'text': 'is', 'char_offset': [17, 18]}, {'text': 'not', 'char_offset': [20, 22]}, {'text': 'known', 'char_offset': [24, 28]}, {'text': 'to', 'char_offset': [30, 31]}, {'text': 'be', 'char_offset': [33, 34]}, {'text': 'involved', 'char_offset': [36, 43]}, {'text': 'in', 'char_offset': [45, 46]}, {'text': 'the', 'char_offset': [48, 50]}, {'text': 'metabolism', 'char_offset': [52, 61]}, {'text': 'of', 'char_offset': [63, 64]}, {'text': 'Plenaxis', 'char_offset': [66, 73]}, {'text': '.', 'char_offset': [74, 74]}]",[<DrugEntity None 11-15 P-450 brand>]
3,DDI-DrugBank.d519.s2,Plenaxis is highly bound to plasma proteins (96 to 99%).,[<DrugEntity DDI-DrugBank.d519.s2.e0 0-7 Plenaxis brand>],[],"[{'text': 'Plenaxis', 'char_offset': [0, 7]}, {'text': 'is', 'char_offset': [9, 10]}, {'text': 'highly', 'char_offset': [12, 17]}, {'text': 'bound', 'char_offset': [19, 23]}, {'text': 'to', 'char_offset': [25, 26]}, {'text': 'plasma', 'char_offset': [28, 33]}, {'text': 'proteins', 'char_offset': [35, 42]}, {'text': '(', 'char_offset': [44, 44]}, {'text': '96', 'char_offset': [45, 46]}, {'text': 'to', 'char_offset': [48, 49]}, {'text': '99', 'char_offset': [51, 52]}, {'text': '%', 'char_offset': [53, 53]}, {'text': ')', 'char_offset': [54, 54]}, {'text': '.', 'char_offset': [55, 55]}]",[<DrugEntity None 35-42 proteins group>]
4,DDI-DrugBank.d519.s3,Laboratory Tests Response to Plenaxis should be monitored by measuring serum total testosterone concentrations just prior to administration on Day 29 and every 8 weeks thereafter.,"[<DrugEntity DDI-DrugBank.d519.s3.e0 29-36 Plenaxis brand>, <DrugEntity DDI-DrugBank.d519.s3.e1 83-94 testosterone drug>]",[<DrugPair DDI-DrugBank.d519.s3.p0 DDI-DrugBank.d519.s3.e0 DDI-DrugBank.d519.s3.e1 None>],"[{'text': 'Laboratory', 'char_offset': [0, 9]}, {'text': 'Tests', 'char_offset': [11, 15]}, {'text': 'Response', 'char_offset': [17, 24]}, {'text': 'to', 'char_offset': [26, 27]}, {'text': 'Plenaxis', 'char_offset': [29, 36]}, {'text': 'should', 'char_offset': [38, 43]}, {'text': 'be', 'char_offset': [45, 46]}, {'text': 'monitored', 'char_offset': [48, 56]}, {'text': 'by', 'char_offset': [58, 59]}, {'text': 'measuring', 'char_offset': [61, 69]}, {'text': 'serum', 'char_offset': [71, 75]}, {'text': 'total', 'char_offset'

In [6]:
df.shape

(5675, 6)

In [7]:
# number of different type of drugs
df.parsed_drugs.apply(len).sum()

12929

### Find differences in parsed and found drugs

In [8]:
def find_diffs(row, diffs=True):
    pd_lst = []
    for parsed_drug in row.parsed_drugs:
        pd_lst.append(parsed_drug.text)
        
    drug_lst = []
    for drug in row.drugs:
        drug_lst.append(drug.text)
    
    if diffs:
        return [item for item in drug_lst if item not in pd_lst]
    else:
        return [item for item in drug_lst if item in pd_lst]

In [9]:
diffs = df.apply(lambda x: find_diffs(x, diffs=True), axis=1)
same = df.apply(lambda x: find_diffs(x, diffs=False), axis=1)

In [10]:
pd_txt = df.parsed_drugs.apply(lambda x: [item.text for item in x])
d_txt = df.drugs.apply(lambda x: [item.text for item in x])

In [11]:
df_sim = pd.concat([pd_txt, d_txt, diffs, same], axis=1)
df_sim.columns = ['parsed_drugs', 'drugs', 'diffs', 'sim']

In [12]:
df_sim.head()

,parsed_drugs,drugs,diffs,sim
0,[],[interactions],[interactions],[]
1,[Plenaxis],[],[],[]
2,[Plenaxis],[P-450],[P-450],[]
3,[Plenaxis],[proteins],[proteins],[]
4,"[Plenaxis, testosterone]","[Tests, testosterone, concentrations]","[Tests, concentrations]",[testosterone]


In [13]:
print('diffs', df_sim.diffs.apply(len).sum())
print('sim', df_sim.sim.apply(len).sum())

diffs 11499
sim 8492


In [14]:
found_drugs = df.apply(lambda x: pd.Series(x['drugs']),axis=1).stack()
print(found_drugs.shape)

(19991,)


### Initial analysis of parsed drugs

In [15]:
drug_ents = df.apply(lambda x: pd.Series(x['parsed_drugs']),axis=1).stack()

In [16]:
drug_ents.head()

1  0       <DrugEntity DDI-DrugBank.d519.s0.e0 45-52 Plenaxis brand>
2  0       <DrugEntity DDI-DrugBank.d519.s1.e0 66-73 Plenaxis brand>
3  0         <DrugEntity DDI-DrugBank.d519.s2.e0 0-7 Plenaxis brand>
4  0       <DrugEntity DDI-DrugBank.d519.s3.e0 29-36 Plenaxis brand>
   1    <DrugEntity DDI-DrugBank.d519.s3.e1 83-94 testosterone drug>
dtype: object

In [17]:
drug_ents.apply(lambda x: x.type).unique()

array(['brand', 'drug', 'group', 'drug_n'], dtype=object)

In [18]:
drugs = drug_ents.apply(lambda x: x.type)

In [19]:
drugs.shape

(12929,)

In [20]:
drugs.value_counts()

drug      8197
group     3206
brand     1423
drug_n     103
dtype: int64

In [21]:
drug_names = drug_ents.apply(lambda x: x.text)

In [22]:
drug_names.describe()

count        12929
unique        2655
top       warfarin
freq           176
dtype: object

In [23]:
drug_types = drug_ents.apply(lambda x: x.type)

In [24]:
drug_types.describe()

count     12929
unique        4
top        drug
freq       8197
dtype: object

In [25]:
drug_chars = drug_ents.apply(lambda x: x.offsets)

In [26]:
df_drugs  = pd.concat([drug_names, drug_types, drug_chars], axis=1).reset_index(drop=True)
df_drugs.columns = ['name','type','offset']

In [27]:
df_drugs.head(11)

,name,type,offset
0,Plenaxis,brand,"[45, 52]"
1,Plenaxis,brand,"[66, 73]"
2,Plenaxis,brand,"[0, 7]"
3,Plenaxis,brand,"[29, 36]"
4,testosterone,drug,"[83, 94]"
5,Plenaxis,brand,"[76, 83]"
6,ORENCIA,brand,"[61, 67]"
7,MTX,drug,"[50, 52]"
8,NSAIDs,group,"[55, 60]"
9,corticosteroids,group,"[63, 77]"


In [28]:
df_drugs.describe()

,name,type,offset
count,12929,12929,12929
unique,2655,4,3939
top,warfarin,drug,"[0, 9]"
freq,176,8197,267


In [29]:
df_drugs.apply(lambda x: x['name'][-3:] if x.type == 'drug' else None, axis=1).value_counts()[:10]

ine    1919
ide     617
cin     396
ole     391
one     330
ate     311
rin     302
ium     291
xin     220
vir     220
dtype: int64

In [30]:
df_drugs.apply(lambda x: x['name'][-4:] if x.type == 'drug' else None, axis=1).value_counts()[:10]

dine    439
zole    378
pine    309
arin    259
line    246
mide    237
mine    227
oxin    215
toin    214
avir    209
dtype: int64

In [31]:
df_drugs.apply(lambda x: x['name'][-5:] if x.type == 'drug' else None, axis=1).value_counts()[:10]

azole    348
idine    311
farin    224
amine    219
navir    206
goxin    205
mycin    204
ytoin    201
thium    148
lline    143
dtype: int64

### Check weird offset cases

In [32]:
#print(df_drugs[df_drugs.offset.str.contains(';')].shape)
#df_drugs[df_drugs.offset.str.contains(';')]

#print(df_drugs.offset.apply(lambda x: x if len(x) >= 2 else None).dropna(how='all').shape)
#df_drugs.offset.apply(lambda x: x if len(x) >= 2 else None).dropna(how='all')